# Func for gigachat API

In [ ]:
import requests
import uuid

def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    rq_uid = str(uuid.uuid4())

    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }


    payload = {
        'scope': scope
    }

    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1
    

def get_chat_completion(auth_token, user_message):
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    payload = json.dumps({
        "model": "GigaChat",
        "messages": [
            {
                "role": "user",
                "content": user_message
            }
        ],
        "temperature": 1,
        "top_p": 0.1,
        "n": 1,
        "stream": False,
        "max_tokens": 10,
        "repetition_penalty": 1, 
        "update_interval": 0
    })

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {auth_token}'
    }

    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Произошла ошибка: {str(e)}")
        return -1


In [ ]:
your_api_token = '<token>'

In [ ]:
response = get_token(your_api_token)
if response != 1:
  print(response.text)
  giga_token = response.json()['access_token']

In [ ]:
prompt = """Ты – эксперт по разметке новостного контента. Твоя задача – определить, к какой одной категории относится данная новость. Ниже приведён список доступных категорий:
0 - Политика: новости о правительстве, выборах, международных отношениях, законодательстве.
1 - Экономика и бизнес: финансовые рынки, корпоративные новости, стартапы, экономические обзоры.
2 - Спорт: результаты матчей, анонсы спортивных событий, интервью, аналитика спортивных событий.
3 - Культура и развлечения: кино, музыка, театр, выставки, обзоры культурных событий.
4 - Технологии и наука: новости IT-сферы, инновации, открытия, гаджеты, исследования.
5 - Здоровье и медицина: новости о медицине, исследования, советы по здоровью, эпидемии.
6 - Общество: социальные тренды, образование, городская жизнь, экология, права человека.
7 - Происшествия и криминал: сообщения о происшествиях, расследованиях, криминальных событиях.
8 - Локальные новости: события, касающиеся определённых регионов или городов.

Инструкция:
Прочитай текст новости, представленный ниже.
Определи основную тему новости, сопоставив её с описаниями категорий.
Выведи в ответ только один класс, соответствующий новости.
Ответ должен состоять из одного слова – названия категории, которое следует после двоеточия.
Например, если новость касается выборов, твой ответ должен быть:
"Политика"

Текст для анализа: /"{text}/". Ответь одни классом из ["Политика","Экономика и бизнес","Спорт","Культура и развлечения","Технологии и наука","Здоровье и медицина","Общество","Происшествия и криминал","Локальные новости"]:"""

In [ ]:
text = 'Чиновники, депутаты и правозащитники вернулись к обсуждению «сухого закона» для водителей. Противники возвращения промилле считают, что в России из-за ментальности населения разрешать пить за рулем даже в минимальных дозах нельзя. С тех пор как обнулили допустимую норму, число «пьяных» ДТП только выросло, а водителей с мизерным опьянением повсеместно лишают прав и вымогают крупные взятки.'
prompt.format(text=text)

In [ ]:
answer = get_chat_completion(giga_token,prompt.format(text=text))
print(answer.json()['choices'][0]['message']['content'])

# Markup datasets from transforrmers

In [4]:
import csv
from tqdm import tqdm
from datasets import load_dataset

name = 'IlyaGusev/gazeta' # the news dataset
dataset = load_dataset(name)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 60964
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6369
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6793
    })
})

In [ ]:
prompt = """Ты – эксперт по разметке новостного контента. Твоя задача – определить, к какой одной категории относится данная новость. Ниже приведён список доступных категорий:
0 - Политика: новости о правительстве, выборах, международных отношениях, законодательстве.
1 - Экономика и бизнес: финансовые рынки, корпоративные новости, стартапы, экономические обзоры.
2 - Спорт: результаты матчей, анонсы спортивных событий, интервью, аналитика спортивных событий.
3 - Культура и развлечения: кино, музыка, театр, выставки, обзоры культурных событий.
4 - Технологии и наука: новости IT-сферы, инновации, открытия, гаджеты, исследования.
5 - Здоровье и медицина: новости о медицине, исследования, советы по здоровью, эпидемии.
6 - Общество: социальные тренды, образование, городская жизнь, экология, права человека.
7 - Происшествия и криминал: сообщения о происшествиях, расследованиях, криминальных событиях.
8 - Локальные новости: события, касающиеся определённых регионов или городов.

Инструкция:
Прочитай текст новости, представленный ниже.
Определи основную тему новости, сопоставив её с описаниями категорий.
Выведи в ответ только один класс, соответствующий новости.
Ответ должен состоять из одного слова – названия категории, которое следует после двоеточия.
Например, если новость касается выборов, твой ответ должен быть:
"Политика"

Текст для анализа: "{text}". Ответь одни классом из ["Политика","Экономика и бизнес","Спорт","Культура и развлечения","Технологии и наука","Здоровье и медицина","Общество","Происшествия и криминал","Локальные новости"]:"""

### Markup only validation set becaus of free token limits

In [ ]:
with open('output2.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['text', 'output'])
    
    for i, data in enumerate(tqdm(dataset['validation'], total=len(dataset['validation']))):
        answer = get_chat_completion(giga_token,prompt.format(text=data['summary'])).json()['choices'][0]['message']['content']
        row = [data['summary'], str(answer)]
        writer.writerow(row)
